# EDA for Big Data Derby Competition

### Creating paths to diferent folders

In [2]:
import os
from pathlib import Path

# Get current work directory of file (Notebook dir)
NOTEBOOK_DIR=Path(os.path.abspath(os.getcwd()))
# Root path of repo
WORK_DIR=NOTEBOOK_DIR.parent.parent
# Path to Datasets
DATA_DIR=WORK_DIR / 'data'

# Noob printing to check if I did this shit right
print("Notebook dir: ", NOTEBOOK_DIR)
print("Repo dir: ", WORK_DIR)
print("Data dir: ", DATA_DIR)

Notebook dir:  r:\kaggle\big-data-derby-kaggle\notebooks\sainth
Repo dir:  r:\kaggle\big-data-derby-kaggle
Data dir:  r:\kaggle\big-data-derby-kaggle\data


### Loading csv into dataframes

In [3]:
import pandas as pd
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
import seaborn as sns

complete_data= pd.read_csv(DATA_DIR / 'nyra_2019_complete.csv', names=['track_id','race_date','race_number','program_number','trakus_index','latitude','longitude','distance_id','course_type','track_condition','run_up_distance','race_type','purse','post_time','weight_carried','jockey','odds','position_at_finish'])
print(complete_data.head())
print(complete_data.dtypes)

C:\Users\santi\AppData\Local\Temp\ipykernel_11972\876585557.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  complete_data= pd.read_csv(DATA_DIR / 'nyra_2019_complete.csv', names=['track_id','race_date','race_number','program_number','trakus_index','latitude','longitude','distance_id','course_type','track_condition','run_up_distance','race_type','purse','post_time','weight_carried','jockey','odds','position_at_finish'])


  track_id   race_date  race_number program_number  trakus_index   latitude  \
0      AQU  2019-01-01            9            6              72  40.672902   
1      AQU  2019-01-01            9            6              73  40.672946   
2      AQU  2019-01-01            9            6              74  40.672990   
3      AQU  2019-01-01            9            6              63  40.672510   
4      AQU  2019-01-01            9            6              64  40.672553   

   longitude  distance_id course_type track_condition  run_up_distance  \
0 -73.827607          600           D             GD                48   
1 -73.827587          600           D             GD                48   
2 -73.827568          600           D             GD                48   
3 -73.827781          600           D             GD                48   
4 -73.827762          600           D             GD                48   

  race_type    purse  post_time  weight_carried                   jockey  \
0   